In [2]:
import numpy as np
import time
import scipy.stats as stats
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
import matplotlib.pyplot as plt
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score)

# Explore and extra data of particular periods of interest

In [3]:
data = pd.read_sas('/home/guangya/Downloads/wnv_2245new.sas7bdat') #Data from week 22 to 45, which is what i used for latter models

In [4]:
data.head() # Available features in the data set, More description is the final.xlsw file

,hexid,Input_FID,tempc,preci,yr,weeks,yrweeks,yrwksfid,templag1,templag2,...,wwpct,ehwpct,yr_hexid,Jantemp,PopYesNo,hpctpreww,hpctpostww,hpct7089,hpctpost90,income1
0,1.0,130.0,16.9758,6.046130,2005.0,22.0,b'200522',b'200522130',15.9957,13.7796,...,27.184466,0.0,"b'2005,1'",-3.815482,1.0,0.0,0.0,0.0,0.0,41.979816
1,1.0,130.0,25.3449,58.968242,2005.0,23.0,b'200523',b'200523130',16.9758,15.9957,...,27.184466,0.0,"b'2005,1'",-3.815482,1.0,0.0,0.0,0.0,0.0,41.979816
2,1.0,130.0,21.0872,15.114200,2005.0,24.0,b'200524',b'200524130',25.3449,16.9758,...,27.184466,0.0,"b'2005,1'",-3.815482,1.0,0.0,0.0,0.0,0.0,41.979816
3,1.0,130.0,22.3485,0.000000,2005.0,25.0,b'200525',b'200525130',21.0872,25.3449,...,27.184466,0.0,"b'2005,1'",-3.815482,1.0,0.0,0.0,0.0,0.0,41.979816
4,1.0,130.0,25.7740,6.371029,2005.0,26.0,b'200526',b'200526130',22.3485,21.0872,...,27.184466,0.0,"b'2005,1'",-3.815482,1.0,0.0,0.0,0.0,0.0,41.979816


In [5]:
data.isna().sum() # Check na

hexid               0
Input_FID           0
tempc               0
preci               0
yr                  0
weeks               0
yrweeks             0
yrwksfid            0
templag1            0
templag2            0
templag3            0
templag4            0
precilag1           0
precilag2           0
precilag3           0
precilag4           0
wnvbinary           0
mirmean             0
year           150510
mirlag1             0
mirlag2             0
mirlag3             0
mirlag4             0
totpop              0
whitepct            0
blackpct            0
asianpct            0
hispanicpct         0
Income              0
owpct               0
dospct              0
dlipct              0
dmipct              0
dhipct              0
blpct               0
dfpct               0
efpct               0
mfpct               0
shrubpct            0
glandpct            0
pasturepct          0
ccpct               0
wwpct               0
ehwpct              0
yr_hexid            0
Jantemp   

In [6]:
data.drop(columns=['wnvbinary','yrweeks','yrwksfid','yr_hexid','year']).isna().sum() # Drop year column so that so na appear

hexid          0
Input_FID      0
tempc          0
preci          0
yr             0
weeks          0
templag1       0
templag2       0
templag3       0
templag4       0
precilag1      0
precilag2      0
precilag3      0
precilag4      0
mirmean        0
mirlag1        0
mirlag2        0
mirlag3        0
mirlag4        0
totpop         0
whitepct       0
blackpct       0
asianpct       0
hispanicpct    0
Income         0
owpct          0
dospct         0
dlipct         0
dmipct         0
dhipct         0
blpct          0
dfpct          0
efpct          0
mfpct          0
shrubpct       0
glandpct       0
pasturepct     0
ccpct          0
wwpct          0
ehwpct         0
Jantemp        0
PopYesNo       0
hpctpreww      0
hpctpostww     0
hpct7089       0
hpctpost90     0
income1        0
dtype: int64

In [7]:
x_total = data.drop(columns=['wnvbinary','yrweeks','yrwksfid','yr_hexid','year']).values # Drop extra column
y_total = data['wnvbinary'].values

In [8]:
x_total = x_total.astype('float64')

In [9]:
# some spot check for data
data[data['hexid'] == 1431]['blackpct'].unique()

array([4.28479215])

In [10]:
data[data['hexid'] == 1831]['whitepct'].unique()

array([86.10261915])

In [11]:
data[data['hexid'] == 1831]['dmipct'].unique() 

array([14.42441054])

In [12]:
data[data['hexid'] == 3121]['income1'].unique() # The Geological and social data is likely a 10 year estimate here, which does not change from 2005-2016

array([100.88401563])

In [13]:
x = data[['yr','templag2','templag3','templag4','precilag2','mirlag1','mirlag2','mirlag3','mirlag4', 'whitepct','owpct','dmipct','dhipct']].values
# Data set for the best model described in paper, table5.
# However, random forest use a different feature selection algorithm 
# so that this might not be the optimal one for oue models. Since it's much slower to train all, I will use ALL features for further optimization later
y = data['wnvbinary'].values 
x = x.astype('float64')

In [14]:
trainX, testX, trainY, testY = train_test_split(x, y, test_size = 0.2, shuffle = True) # CV

In [15]:
trainX_total, testX_total, trainY_total, testY_total = train_test_split(x_total, y_total, test_size = 0.2, shuffle = True) # CV for all

# Test on Random Forest

In [59]:
def model_RF_test(model_RF, dataX, dataY):
    print("Model performance")
    predict_data = model_RF.predict_proba(dataX)
    
    # Some stats
    print("Feature Importantce : ")
    print(model_RF.feature_importances_)
    print("Total number of WNV occurence in test set : " + str(len(dataY[dataY > 0])))
    
    print("Number of WNV occurence the model is able to capture in test set:" + str(dataY[np.where(predict_data[:,1]  > 0)].sum()))
    
    print("Expected number of WNV occurence of models aussume prediction is normally distributed : " + str(predict_data[:,1].mean() * len(predict_data)))
    print("Log loss : " + str(log_loss(dataY,predict_data)))
    
    print("This is to test the performance of random forest model, ideally, the logloss is low and also it is able to capture most of the WNV occurence, and also it is neither over predicting" +  
          "nor under predicting")
    
    return None # Check how many wnv it predicts

In [17]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(trainX_total, trainY_total)
time_start = time.time()
model_RF1 = RandomForestClassifier(n_estimators=400,
                                 n_jobs = -1,
                                 max_features=None,
                                 max_depth= None,
                                 bootstrap=True,
                                class_weight='balanced'
                                 ) # Use undersampling to see if it worked
model_RF1.fit(X_resampled, y_resampled)
print("time consumed:", time.time() - time_start) 

Using TensorFlow backend.


time consumed: 3.264770030975342


In [60]:
model_RF_test(model_RF1,testX_total,testY_total) # The result tend to predict a lot of 1s, which is very biased.

Model performance
Feature Importantce : 
[0.00999531 0.01067965 0.01496773 0.01082702 0.0054172  0.02645585
 0.02794946 0.02833401 0.03969041 0.03359853 0.01333262 0.01407395
 0.01410982 0.01532784 0.02686748 0.13324849 0.11390923 0.11554932
 0.01995433 0.13395407 0.00646951 0.00946079 0.00790575 0.00963136
 0.00768387 0.00252334 0.01573296 0.01404953 0.01017898 0.01184233
 0.00080315 0.0024459  0.0021979  0.00341815 0.00042789 0.00192679
 0.00188718 0.00171914 0.00172571 0.00063831 0.02298358 0.
 0.01426518 0.01014777 0.01229566 0.01120487 0.00819208]
Total number of WNV occurence in test set : 173
Number of WNV occurence the model is able to capture in test set:173.0
Expected number of WNV occurence of models aussume prediction is normally distributed : 69593.33
Log loss : 0.4248442677492819
This is to test the performance of random forest model, ideally, the logloss is low and also it is able to capture most of the WNV occurence, and also it is neither over predictingnor under predi

In [19]:
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(trainX, trainY)
time_start = time.time()
model_RF2 = RandomForestClassifier(n_estimators=400,
                                 n_jobs = -1,
                                 max_features=None,
                                 max_depth= None,
                                 bootstrap=True,
                                class_weight='balanced'
                                 ) # Use undersampling to see if it worked
model_RF2.fit(X_resampled, y_resampled)
print("time consumed:", time.time() - time_start) 

time consumed: 1.166808843612671


In [61]:
model_RF_test(model_RF2,testX,testY) 

Model performance
Feature Importantce : 
[0.02332565 0.07673163 0.09381062 0.08244171 0.04115731 0.09283884
 0.32511096 0.03987311 0.05722872 0.04018178 0.02340537 0.06687365
 0.03702065]
Total number of WNV occurence in test set : 196
Number of WNV occurence the model is able to capture in test set:195.0
Expected number of WNV occurence of models aussume prediction is normally distributed : 73071.56249999999
Log loss : 0.42897848898311103
This is to test the performance of random forest model, ideally, the logloss is low and also it is able to capture most of the WNV occurence, and also it is neither over predictingnor under predicting


In [33]:
time_start = time.time()
model_RF3 = RandomForestClassifier(n_estimators=200,
                                 n_jobs = -1,
                                 max_features='sqrt',
                                 max_depth= 20,
                                 bootstrap=True,
                                   min_samples_leaf= 2,
                                class_weight='balanced'
                                 ) # Try more trees and see if calibration curve gets better
model_RF3.fit(trainX_total, trainY_total)
print("time consumed:", time.time() - time_start) 

time consumed: 307.3307168483734


In [88]:
x = sorted(model_RF3.feature_importances_)[::-1]
seq = [list(model_RF3.feature_importances_).index(v) for v in x]

In [91]:
seq[:10]

[15, 16, 17, 19, 8, 14, 18, 9, 7, 5]

In [57]:
model_RF_test(model_RF3,testX_total,testY_total)

Model performance
Feature Importantce : 
[1.30942271e-02 1.28020934e-02 2.92368487e-02 1.19465584e-02
 7.07868054e-03 4.19247572e-02 2.91652172e-02 4.69769293e-02
 6.08182076e-02 4.90170180e-02 1.13150864e-02 1.15022790e-02
 1.32276167e-02 1.27103614e-02 5.50808196e-02 9.72655123e-02
 7.87749447e-02 7.80864630e-02 5.01764878e-02 7.08532494e-02
 8.85207640e-03 9.74098519e-03 8.80785359e-03 1.12414583e-02
 9.87450040e-03 6.71593176e-03 1.24806795e-02 1.14679958e-02
 1.23319782e-02 1.02753625e-02 1.17618205e-03 8.00185418e-03
 9.13261608e-04 3.29746529e-03 4.22286487e-04 4.30764953e-03
 1.68786058e-03 9.07328043e-04 4.42005112e-03 7.72870641e-05
 2.79016966e-02 0.00000000e+00 1.64946315e-02 9.70867099e-03
 1.27331784e-02 1.49233092e-02 1.01851078e-02]
Total number of WNV occurence in test set : 173
Number of WNV occurence the model is able to capture in test set:165.0
Expected number of WNV occurence of models aussume prediction is normally distributed : 13746.057522374354
Log loss : 0.06

In [102]:
data.drop(columns=['wnvbinary','yrweeks','yrwksfid','yr_hexid','year']).columns

Index(['hexid', 'Input_FID', 'tempc', 'preci', 'yr', 'weeks', 'templag1',
       'templag2', 'templag3', 'templag4', 'precilag1', 'precilag2',
       'precilag3', 'precilag4', 'mirmean', 'mirlag1', 'mirlag2', 'mirlag3',
       'mirlag4', 'totpop', 'whitepct', 'blackpct', 'asianpct', 'hispanicpct',
       'Income', 'owpct', 'dospct', 'dlipct', 'dmipct', 'dhipct', 'blpct',
       'dfpct', 'efpct', 'mfpct', 'shrubpct', 'glandpct', 'pasturepct',
       'ccpct', 'wwpct', 'ehwpct', 'Jantemp', 'PopYesNo', 'hpctpreww',
       'hpctpostww', 'hpct7089', 'hpctpost90', 'income1'],
      dtype='object')

In [35]:
time_start = time.time()
model_RF4 = RandomForestClassifier(n_estimators=200,
                                 n_jobs = -1,
                                 max_features='sqrt',
                                 max_depth= 20,
                                 bootstrap=True,
                                   min_samples_leaf= 2,
                                class_weight='balanced'
                                 ) # Try more trees and see if calibration curve gets better
model_RF4.fit(trainX, trainY)
print("time consumed:", time.time() - time_start) 

time consumed: 218.0876429080963


In [62]:
model_RF_test(model_RF4,testX,testY)

Model performance
Feature Importantce : 
[0.02621741 0.08807811 0.09189799 0.09668619 0.03990629 0.14748003
 0.14935428 0.11992112 0.08589193 0.03846456 0.02059316 0.057597
 0.03791192]
Total number of WNV occurence in test set : 196
Number of WNV occurence the model is able to capture in test set:181.0
Expected number of WNV occurence of models aussume prediction is normally distributed : 13670.897052778864
Log loss : 0.06785630091527631
This is to test the performance of random forest model, ideally, the logloss is low and also it is able to capture most of the WNV occurence, and also it is neither over predictingnor under predicting


In [105]:
data[['yr','templag2','templag3','templag4','precilag2','mirlag1','mirlag2','mirlag3','mirlag4', 'whitepct','owpct','dmipct','dhipct']].columns

Index(['yr', 'templag2', 'templag3', 'templag4', 'precilag2', 'mirlag1',
       'mirlag2', 'mirlag3', 'mirlag4', 'whitepct', 'owpct', 'dmipct',
       'dhipct'],
      dtype='object')

In [104]:
data.drop(columns=['wnvbinary','yrweeks','yrwksfid','yr_hexid','year']).columns[[17,20,9,10,16,18]]West Nile Virus Chicago: Wayne

Index(['mirlag3', 'whitepct', 'templag4', 'precilag1', 'mirlag2', 'mirlag4'], dtype='object')

In [96]:
data[data.drop(columns=['wnvbinary','yrweeks','yrwksfid','yr_hexid','year']).columns[[7,8,9,10,15,16,17,20]]].head() # From the above 3 feature importance of models, we can manually select 
# about 7 features which is mostly important

,templag2,templag3,templag4,precilag1,mirlag1,mirlag2,mirlag3,whitepct
0,13.7796,16.7306,9.0730,0.019091,0.00000,0.00000,0.00000,65.74257
1,15.9957,13.7796,16.7306,6.046130,0.00000,0.00000,0.00000,65.74257
2,16.9758,15.9957,13.7796,58.968242,2.50278,0.00000,0.00000,65.74257
3,25.3449,16.9758,15.9957,15.114200,0.00000,2.50278,0.00000,65.74257
4,21.0872,25.3449,16.9758,0.000000,0.00000,0.00000,2.50278,65.74257


In [97]:
x_selected = data[data.drop(columns=['wnvbinary','yrweeks','yrwksfid','yr_hexid','year']).columns[[7,8,9,10,15,16,17,20]]]
y_selected = y

### From the above model, we can see that random forest, although is not very good, can actually capture someinformation. So we will try formal Cross validation on selected features and see if it gets betterz

## Pipnelines for later for wrok

In [98]:
trainX_sel, testX_sel, trainY_sel, testY_sel = train_test_split(x_selected.values, y_selected, test_size = 0.2, shuffle = True) # CV

## Find best model 1

In [39]:
time_start = time.time()
params_RF_grid_1 = {
    'n_estimators' : [500, 1000],
    'max_features' : [90, 'sqrt', None],
    'max_depth' : [10, None],
    'min_samples_leaf' : [1,2]
}
CV_model_RF_1 = GridSearchCV(model_RF, params_RF_grid_1, scoring='neg_log_loss',cv=5)
CV_model_RF_1.fit(x_selected, dataY)
print("time consumed:", time.time() - time_start)

time consumed: 6569.04248213768


In [40]:
CV_model_RF_1.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [41]:
CV_model_RF_1.best_params_

{'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'n_estimators': 1000}

In [99]:
time_start = time.time()
model_RF_best_1 = RandomForestClassifier(n_estimators=800,
                                 n_jobs = -1,
                                 max_features="sqrt",
                                 max_depth=None,
                                 bootstrap=True,
                                 min_samples_leaf=2
                                 )
model_RF_best_1.fit(trainX_sel,trainY_sel)
print("Time consumed:", time.time() - time_start)

Time consumed: 974.386381149292


In [100]:
model_RF_test(model_RF_best_1,testX_sel,testY_sel)

Model performance
Feature Importantce : 
[0.13570302 0.13524317 0.13471302 0.12046497 0.12085287 0.12269303
 0.12148794 0.10884199]
Total number of WNV occurence in test set : 174
Number of WNV occurence the model is able to capture in test set:128.0
Expected number of WNV occurence of models aussume prediction is normally distributed : 185.4557672240931
Log loss : 0.008748338479888089
This is to test the performance of random forest model, ideally, the logloss is low and also it is able to capture most of the WNV occurence, and also it is neither over predictingnor under predicting


## Find best model 2

In [ ]:
time_start = time.time()
params_RF_grid_2 = {
    'n_estimators' : [800, 1200],
    'max_features' : ['sqrt', 5],
    'min_samples_leaf' : [2,3]
}
CV_model_RF_2 = GridSearchCV(model_RF, params_RF_grid_2, cv=5)
CV_model_RF_2.fit(dataX, dataY)
print("time consumed:", time.time() - time_start)

In [24]:
CV_model_RF_2.fit(dataX, dataY)

KeyboardInterrupt: 

In [ ]:
CV_model_RF_2.best_params_

In [11]:
time_start = time.time()
model_RF_best_2 = RandomForestRegressor(n_estimators=1500,
                                 criterion="mse",
                                 n_jobs = -1,
                                 max_features=None,
                                 max_depth=None,
                                 bootstrap=True,
                                 min_samples_leaf=2
                                 )
model_RF_best_2.fit(trainX, trainY)
print("Time consumed:", time.time() - time_start)

Time consumed: 263.9204020500183


## Find best model 3

In [12]:
time_start = time.time()
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, None, 110],
    'max_features': ['sqrt', 4],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [500, 2000, 4000, 1000]
}
CV_model_RF_3 = GridSearchCV(model_RF_best_2, params_RF_grid_3, cv=5)
CV_model_RF_3.fit(dataX, dataY)
print("time consumed:", time.time() - time_start)

time consumed: 998.9455091953278


In [13]:
CV_model_RF_3.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='log2', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=3000,
                      n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [14]:
CV_model_RF_3.best_params_

{'max_features': 'log2', 'min_samples_leaf': 2, 'n_estimators': 3000}

In [51]:
time_start = time.time()
model_RF_best_3 = RandomForestRegressor(n_estimators=8000,
                                 criterion="mse",
                                 n_jobs = -1,
                                 max_features="log2",
                                 max_depth=None,
                                 bootstrap=True,
                                 min_samples_leaf=2
                                 )
model_RF_best_3.fit(trainX, trainY)
print("Time consumed:", time.time() - time_start)

Time consumed: 45.644662857055664
